# Google Colab으로 오픈소스 LLM 구동하기

## 1단계 - LLM 양자화에 필요한 패키지 설치
- bitsandbytes: Bitsandbytes는 CUDA 사용자 정의 함수, 특히 8비트 최적화 프로그램, 행렬 곱셈(LLM.int8()) 및 양자화 함수에 대한 경량 래퍼
- PEFT(Parameter-Efficient Fine-Tuning): 모델의 모든 매개변수를 미세 조정하지 않고도 사전 훈련된 PLM(언어 모델)을 다양한 다운스트림 애플리케이션에 효율적으로 적용 가능
- accelerate: PyTorch 모델을 더 쉽게 여러 컴퓨터나 GPU에서 사용할 수 있게 해주는 도구


In [43]:
#양자화에 필요한 패키지 설치
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## 2단계 - 트랜스포머에서 BitsandBytesConfig를 통해 양자화 매개변수 정의하기


* load_in_4bit=True: 모델을 4비트 정밀도로 변환하고 로드하도록 지정
* bnb_4bit_use_double_quant=True: 메모리 효율을 높이기 위해 중첩 양자화를 사용하여 추론 및 학습
* bnd_4bit_quant_type="nf4": 4비트 통합에는 2가지 양자화 유형인 FP4와 NF4가 제공됨. NF4 dtype은 Normal Float 4를 나타내며 QLoRA 백서에 소개되어 있습니다. 기본적으로 FP4 양자화 사용
* bnb_4bit_compute_dype=torch.bfloat16: 계산 중 사용할 dtype을 변경하는 데 사용되는 계산 dtype. 기본적으로 계산 dtype은 float32로 설정되어 있지만 계산 속도를 높이기 위해 bf16으로 설정 가능



In [44]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## 3단계 - 경량화 모델 로드하기

이제 모델 ID를 지정한 다음 이전에 정의한 양자화 구성으로 로드합니다.

In [45]:
model_id = "kyujinpy/Ko-PlatYi-6B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [46]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(78464, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
  

## 4단계 - 잘 실행되는지 확인

In [47]:
device = "cuda:0"

messages = [
    {"role": "user", "content": "뉴턴과 아인슈타인이 서로 팔씨름을 하면 누가 이길지 추론해줘"}
]


encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)


generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<|startoftext|> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

뉴턴과 아인슈타인이 서로 팔씨름을 하면 누가 이길지 추론해줘 [/INST]<|endoftext|>


## 5단계- RAG 시스템 결합하기

In [48]:
# pip install시 utf-8, ansi 관련 오류날 경우 필요한 코드
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [49]:
!pip install kaleido python-multipart


In [50]:
!pip install typing-extensions==4.5.0


  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.30 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
fastapi 0.111.0 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.
pydantic 2.7.3 requires typing-extensions>=4.6.1, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.18.4 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
torch 2.3.0+cu121 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incom

In [51]:
!pip -q install langchain pypdf chromadb sentence-transformers faiss-gpu

In [52]:
!pip install -U langchain-community

In [53]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=300,
)

prompt_template = """
### [INST]
Instruction: Answer the question based on your knowledge.
Here is context to help:

{context}

### QUESTION:
{question}

[/INST]
 """

koplatyi_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=koplatyi_llm, prompt=prompt)

In [54]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
loader = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/ESTsoft - The most universal interface for human.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

db = FAISS.from_documents(texts, hf)
retriever = db.as_retriever(
                            search_type="similarity",
                            search_kwargs={'k': 3}
                        )

In [72]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)


In [73]:
import warnings
warnings.filterwarnings('ignore')

In [75]:
result = rag_chain.invoke("이스트소프트의 핵심가치는?")

for i in result['context']:
    print(f"주어진 근거: {i.page_content} / 출처: {i.metadata['source']} - {i.metadata['page']} \n\n")

print(f"\n답변: {result['text']}")

주어진 근거: 임직원(명)
173
이스트소프트는 차별화된 인공지능 기술을  
주축으로 크게 
AI HUMAN,  COMMERCE,  TECHFIN,  
SECURITY , GAME 5가지 사업을 추진하
고 있습니다.
각 분야에 기술적 혁신을 창출하여 AI를 통
한 PLUS된 서비스 경험을 만들어 내고 있
습니다.
핵심가치
잘하는 일을 합니다.
우리는 각자 자기 자신이 무엇을 잘 하는지 알고 있습니다.  내가 잘하는 일을 하여 성과를 내고 보람을 느낍니다.
부끄럽지 않은 일을 합니다.
빠른 길이 바른 길이 아니라면 가지 않습니다.  선을 지키는 올바른 길을 선택합니다.
익숙함에 익숙해지지 않습니다.
관성을 경계합니다.   낯선 것을 받아들이며 배우고 발전시키는 도전을 즐깁니다. / 출처: /content/drive/MyDrive/Colab Notebooks/ESTsoft - The most universal interface for human.pdf - 1 


주어진 근거: AI 기술을  바탕으로  한  최고의  서비스  기업
이스트소프트  입니다 .
Reimagining y our e veryday life wit h AI Human
AI Human Lab 회사소개 사업소개 연혁 윤리경영 공지사항
보다 편리하고 , 안전한  세상을  위한  
실용주의 ( 實用主義  인공지 능  
바로   이스 트소프트 가   지 향 하 는   가 치입니 다  
우리가 제공하는 실용주의 인공지능은 서비스나 제품으로 사용화하는데 오랜 시간이 걸리는 미래의 것이 아닙니다.  
지금 바로 우리의 일상에 인공지능 기술을 더해 업무와 생활에 편의를 높일 수 있는 기술 즉,  'AI PLUS'입니다.  
더 편리한 우리의 일상,  더 안전한 세상을 만들기 위해 노력하는 것,  이것이 바로 이스트소프트가 만들어 가는 실용주의 인공지능입니
다. 
이스트소프트는 고도화된 인공지능으로 더욱 안전하고 편리한 세상을 만들어 나가겠습니다.
설립일
1993. 10. 02
자본금(원)
55.8억 / 출처